In [30]:
import cv2 as cv
import numpy as np

In [31]:
def trackObjects(frame, colorSpace="HSV"):
    '''Given a frame of video and the corresponding color space, find green and
    red objects, drawing bounding boxes around them. Return the modified frame.
    '''

    # Color constants depending on color-space
    # In HSV for better specificity
    redBound = [0,255,255]
    greenBound = [60,255,255]
    redBotLow = np.uint8([0,50,50])
    redBotHigh = np.uint8([10,255,255])
    redTopLow = np.uint8([170,50,50])
    redTopHigh = np.uint8([180,255,255])
    greenLow = np.uint8([45,50,50])
    greenHigh = np.uint8([75,255,255])
    if colorSpace == "BGR":
        redBound = cv.cvtColor(np.uint8([[redBound]]), cv.COLOR_HSV2BGR)[0,0].tolist()
        greenBound = cv.cvtColor(np.uint8([[greenBound]]), cv.COLOR_HSV2BGR)[0,0].tolist()
        redBotLow = cv.cvtColor(np.uint8([[redBotLow]]), cv.COLOR_HSV2BGR)[0,0]
        redBotHigh = cv.cvtColor(np.uint8([[redBotHigh]]), cv.COLOR_HSV2BGR)[0,0]
        redTopLow = cv.cvtColor(np.uint8([[redTopLow]]), cv.COLOR_HSV2BGR)[0,0]
        redTopHigh = cv.cvtColor(np.uint8([[redTopHigh]]), cv.COLOR_HSV2BGR)[0,0]
        greenLow = cv.cvtColor(np.uint8([[greenLow]]), cv.COLOR_HSV2BGR)[0,0]
        greenHigh = cv.cvtColor(np.uint8([[greenHigh]]), cv.COLOR_HSV2BGR)[0,0]


    # Grab red and green objects separately
    redBotObjects = cv.inRange(frame, redBotLow, redBotHigh)
    redTopObjects = cv.inRange(frame, redTopLow, redTopHigh)
    redObjects = cv.bitwise_or(redBotObjects, redTopObjects)
    greenObjects = cv.inRange(frame, greenLow, greenHigh)

    # Find contours and draw bounding boxes on original frame for each
    redContours, _ = cv.findContours(redObjects,
                                    cv.RETR_LIST,
                                    cv.CHAIN_APPROX_SIMPLE)
    greenContours, _ = cv.findContours(greenObjects,
                                    cv.RETR_LIST,
                                    cv.CHAIN_APPROX_SIMPLE)

    # If we found contours, pick out contours with largest area from each set,
    # draw them on the frame, and return out their centers
    # TODO: Take all contours with area greater than a threshold to get dots
    drawnFrame = frame
    redCenter = greenCenter = None
    if redContours:
        largestRed = max(redContours, key=compCnt)
        drawnFrame, redCenter = drawContour(drawnFrame, largestRed, redBound)
    if greenContours:
        largestGreen = max(greenContours, key=compCnt)
        drawnFrame, greenCenter = drawContour(drawnFrame, largestGreen, greenBound)

    return drawnFrame, redCenter, greenCenter

# Key function used to compare contours
def compCnt(cnt):
        return cv.contourArea(cnt)

# Draws enclosing circle for given contour, returns center
def drawContour(frame, cnt, color):
    newFrame = frame
    (x,y),radius = cv.minEnclosingCircle(cnt)
    center = (int(x),int(y))
    radius = int(radius)
    newFrame = cv.circle(newFrame, center, radius, color, 2)
    return newFrame, center

In [ ]:
def inBoundary(boundary, point):
    return cv.pointPolygonTest(boundary, point, False) >= 0

In [ ]:
def trackAndDrawCar(frame, boundary, dots, collected):
    '''Tracks the largest red object, representing the car. Checks if the car is
    outside the boundary and if it is in collecting range of any uncollected
    dots. Reacts accordingly and returns a tuple of (newFrame, inBoundary).'''
    carContour = trackCar(frame)
    M = cv.moments(carContour)
    carCenter = (int(M['m10']/M['m00']), int(M['m01']/M['m00']))
    checkCollections(carContour, dots, collected)
    
    return (newFrame, inBoundary(boundary, carCenter))
    

def trackCar(frame):
    '''Tracks the largest red object, returning the contour.'''
    redBotLow = np.uint8([0,50,50])
    redBotHigh = np.uint8([10,255,255])
    redTopLow = np.uint8([170,50,50])
    redTopHigh = np.uint8([180,255,255])
    
    redBotObjects = cv.inRange(frame, redBotLow, redBotHigh)
    redTopObjects = cv.inRange(frame, redTopLow, redTopHigh)
    redObjects = cv.bitwise_or(redBotObjects, redTopObjects)

    redContours, _ = cv.findContours(redObjects,
                                    cv.RETR_LIST,
                                    cv.CHAIN_APPROX_SIMPLE)
    return max(redContours, key=lambda c: cv.contourArea(c))

def checkCollections(carContour, dots, collected):
    '''Check if the car has collected any dots this frame. A car collects a dot
    if the dot's center is in the car's contour.'''
    for dot in dots:
        if inBoundary(carContour, dot[0]):
            dots.remove(dot)
            collected.add(dot)
            

In [34]:
def parseGameArea(frame, threshold, colorSpace="HSV"):
    '''Setup function that finds the static boundary as well as the centers of
    all dots. Returns the approximation of the boundary along with a list of dot
    centers. Intended to only run once on startup unless we fail to capture the
    boundary.'''

    boundary = findBoundary(frame, colorSpace)
    dots = findDots(frame, threshold, colorSpace)
    return boundary, dots

def findDots(frame, threshold, colorSpace="HSV"):
    '''Given a frame of a video, look for all green dots larger than a
    particular threshold and return their min bounding circles.'''

    # Color constants depending on color-space
    # In HSV for better specificity
    greenLow = np.uint8([45,50,50])
    greenHigh = np.uint8([75,255,255])
    if colorSpace == "BGR":
        greenLow = cv.cvtColor(np.uint8([[greenLow]]), cv.COLOR_HSV2BGR)[0,0]
        greenHigh = cv.cvtColor(np.uint8([[greenHigh]]), cv.COLOR_HSV2BGR)[0,0]

    # Get all green objects larger than a threshold
    greenObjects = cv.inRange(frame, greenLow, greenHigh)
    greenContours, _ = cv.findContours(greenObjects,
                                    cv.RETR_LIST,
                                    cv.CHAIN_APPROX_SIMPLE)
    filteredContours = filter(lambda c: cv.contourArea(c) > threshold, greenContours)

    # Return set of bounding circles. Of the format [((x,y),radius)...]
    circles = set([cv.minEnclosingCircle(c) for c in filteredContours])
    return circles

def findBoundary(frame, colorSpace="HSV"):
    '''Given a frame of a video, look for the blue boundary, make an
    approximation of its outline, and return it.'''

    # Color constants
    purpleBound = [150,255,255]
    blueLow = np.uint8([100,50,50])
    blueHigh = np.uint8([130,255,255])
    if colorSpace == "BGR":
        purpleBound = cv.cvtColor(np.uint8([[purpleBound]]), cv.COLOR_HSV2BGR)[0,0].tolist()
        blueLow = cv.cvtColor(np.uint8([[blueLow]]), cv.COLOR_HSV2BGR)[0,0]
        blueHigh = cv.cvtColor(np.uint8([[blueHigh]]), cv.COLOR_HSV2BGR)[0,0]

    drawnFrame = frame

    # Grab biggest blue object
    blueObjects = cv.inRange(frame, blueLow, blueHigh)
    blueContours, _ = cv.findContours(blueObjects,
                                cv.RETR_LIST,
                                cv.CHAIN_APPROX_NONE)
    if blueContours:
        largestBlue = max(blueContours, key=compCnt)

        # Find outline
        epsilon = 0.01 * cv.arcLength(largestBlue, True)
        outline = cv.approxPolyDP(largestBlue, epsilon, True)
        return outline

    # If we couldn't find the boundary, return None so we try again on the next
    # frame
    return None

In [35]:
def run(threshold, colorSpace="HSV"):
    '''Starts up and runs the camera, passing frames to the appropriate computer
    vision functions and displaying the resulting frame.'''

    cap = cv.VideoCapture(1)

    boundary = None
    dots = None
    collected = set()
    while True:
        # Take each frame
        _, frame = cap.read()

        # Convert BGR to HSV, if necessary
        if colorSpace == "HSV":
            frame = cv.cvtColor(frame, cv.COLOR_BGR2HSV)

        # Process frame
        if not boundary:
            boundary, dots = parseGameArea(frame, threshold, colorSpace)
        finalFrame, redCenter, greenCenter = trackObjects(frame, colorSpace)

        # Convert back to BGR, if necessary
        if colorSpace == "HSV":
            finalFrame = cv.cvtColor(finalFrame, cv.COLOR_HSV2BGR)

        # Display new frame
        cv.imshow("bounded", finalFrame)

        # Stop when ESC is pressed
        k = cv.waitKey(5) & 0xFF
        if k == 27:
            break

    cv.destroyAllWindows()

In [36]:
run(200)